# Initialization

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [4]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 30
OTHER_DATE = '2023-10-27'
_RESETS = True
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


# Dev get IQ data

import numpy as np

SHOTS = int(1e3)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]


stim_circuit = simulator.get_stim_circuit(NOISE_LIST)
meas_outcomes = stim_circuit.compile_sampler(seed=42).sample(SHOTS)
print("generated counts")

if not simulator._resets:
    # LOGIC
    pass

IQ_memory = np.zeros_like(meas_outcomes, dtype=np.complex128)
len_IQ_array = len(simulator.qubit_mapping)
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}    

print("starting kde samples needed")
for row in meas_outcomes:
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        kde_samples_needed[qubit_idx][str(int(bit))] += 1

print("starting kde samples")
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = simulator.kde_dict[qubit_idx], simulator.scaler_dict[qubit_idx]
    if needed_nb_samples['0'] > 0:
        samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42))
    else:
        samples0 = np.empty((0, 2)) 
    if needed_nb_samples['1'] > 0:
        samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42))
    else:
        samples1 = np.empty((0, 2)) 
    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}

print("starting IQ memory")
for row_idx, row in enumerate(meas_outcomes):
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        sample_index = sample_counters[qubit_idx][str(int(bit))]
        sample = kde_samples[qubit_idx][str(int(bit))][sample_index]   
        IQ_memory[row_idx, IQ_idx] = complex(sample[0], sample[1])
        sample_counters[qubit_idx][str(int(bit))] += 1

assert sample_counters == kde_samples_needed

print(IQ_memory.shape)
IQ_memory

import numpy as np
import time

SHOTS = int(1e3)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2]  # [two-qubit-fidelity, reset error, measurement error, idle error]

start_time = time.time()
stim_circuit = simulator.get_stim_circuit(NOISE_LIST)
meas_outcomes = stim_circuit.compile_sampler(seed=42).sample(SHOTS)
print("generated counts", "Time taken:", time.time() - start_time, "seconds")

if not simulator._resets:
    # LOGIC
    pass

IQ_memory = np.zeros_like(meas_outcomes, dtype=np.complex128)

len_IQ_array = len(simulator.qubit_mapping)
used_qubits = set(simulator.qubit_mapping.values())
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}    

start_time = time.time()  # Start timing
for IQ_idx in range(len_IQ_array):
    qubit_idx = simulator.qubit_mapping[IQ_idx]
    outcomes = meas_outcomes[:, IQ_idx]
    counts = np.bincount(outcomes, minlength=2)
    kde_samples_needed[qubit_idx]['0'] += counts[0]
    kde_samples_needed[qubit_idx]['1'] += counts[1]
print("kde samples needed calculated", "Time taken:", time.time() - start_time, "seconds")

start_time = time.time()  # Start timing
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = simulator.kde_dict[qubit_idx], simulator.scaler_dict[qubit_idx]
    samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42)) if needed_nb_samples['0'] > 0 else np.empty((0, 2))
    samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42)) if needed_nb_samples['1'] > 0 else np.empty((0, 2))
    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}
print("kde samples generated", "Time taken:", time.time() - start_time, "seconds")

start_time = time.time()  # Start timing
next_sample_indices = np.zeros((len(simulator.kde_dict), 2), dtype=int)
for row_idx, row in enumerate(meas_outcomes):
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        sample_index = next_sample_indices[qubit_idx, int(bit)]
        sample = kde_samples[qubit_idx][str(int(bit))][sample_index]
        IQ_memory[row_idx, IQ_idx] = complex(sample[0], sample[1])
        next_sample_indices[qubit_idx, int(bit)] += 1
print("IQ memory filled", "Time taken:", time.time() - start_time, "seconds")

print(IQ_memory.shape)


In [33]:
import numpy as np
import time

start_time = time.time()

SHOTS = int(1e5)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]

stim_circuit = simulator.get_stim_circuit(NOISE_LIST)
meas_outcomes = stim_circuit.compile_sampler(seed=42).sample(SHOTS)

elapsed_time = time.time() - start_time
print("generated counts", "Time taken:", elapsed_time, "seconds")

start_time = time.time()
if not simulator._resets:
    # LOGIC
    pass
elapsed_time = time.time() - start_time
print("NO RESET logic applied", "Time taken:", elapsed_time, "seconds")

start_time = time.time()

IQ_memory = np.zeros_like(meas_outcomes, dtype=np.complex128)

len_IQ_array = len(simulator.qubit_mapping)
used_qubits = set(simulator.qubit_mapping.values())
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}    

print("starting kde samples needed", "Time taken to initialize variables:", time.time() - start_time, "seconds")

start_time = time.time()
for IQ_idx in range(len_IQ_array):
    qubit_idx = simulator.qubit_mapping[IQ_idx]
    outcomes = meas_outcomes[:, IQ_idx]
    counts = np.bincount(outcomes, minlength=2)
    kde_samples_needed[qubit_idx]['0'] += counts[0]
    kde_samples_needed[qubit_idx]['1'] += counts[1]

elapsed_time = time.time() - start_time
print("kde samples needed calculated", "Time taken:", elapsed_time, "seconds")

start_time = time.time()
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = simulator.kde_dict[qubit_idx], simulator.scaler_dict[qubit_idx]
    if needed_nb_samples['0'] > 0:
        samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42))
    else:
        samples0 = np.empty((0, 2)) 
    if needed_nb_samples['1'] > 0:
        samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42))
    else:
        samples1 = np.empty((0, 2)) 
    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}

elapsed_time = time.time() - start_time
print("kde samples generated", "Time taken:", elapsed_time, "seconds")

start_time = time.time()
next_sample_indices = np.zeros((len(simulator.kde_dict), 2), dtype=int)
sample_indices_matrix = np.zeros_like(meas_outcomes, dtype=int)
current_sample_indices = np.zeros((len(simulator.kde_dict), 2), dtype=int)

for row_idx, row in enumerate(meas_outcomes):
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        bit_int = int(bit)
        sample_indices_matrix[row_idx, IQ_idx] = current_sample_indices[qubit_idx, bit_int]
        current_sample_indices[qubit_idx, bit_int] += 1

for row_idx in range(meas_outcomes.shape[0]):
    for IQ_idx in range(meas_outcomes.shape[1]):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        bit_int = int(meas_outcomes[row_idx, IQ_idx])
        sample_index = sample_indices_matrix[row_idx, IQ_idx]
        sample = kde_samples[qubit_idx][str(bit_int)][sample_index]
        IQ_memory[row_idx, IQ_idx] = complex(sample[0], sample[1])

elapsed_time = time.time() - start_time
print("IQ memory filled", "Time taken:", elapsed_time, "seconds")

print("IQ memory shape:", IQ_memory.shape)


generated counts Time taken: 0.07734799385070801 seconds
NO RESET logic applied Time taken: 2.8848648071289062e-05 seconds
starting kde samples needed Time taken to initialize variables: 0.010066986083984375 seconds
kde samples needed calculated Time taken: 0.026732921600341797 seconds
kde samples generated Time taken: 0.43048906326293945 seconds
IQ memory filled Time taken: 9.71604609489441 seconds
IQ memory shape: (100000, 63)


In [30]:
max_samples_0 = max(len(samples['0']) for samples in kde_samples.values())
max_samples_1 = max(len(samples['1']) for samples in kde_samples.values())
max_samples = max(max_samples_0, max_samples_1)

num_qubits = len(kde_samples)

aligned_samples = np.full((num_qubits, 2, max_samples, 2), fill_value=np.nan)

for qubit_idx, outcomes in kde_samples.items():
    for outcome_str, samples in outcomes.items():
        outcome = int(outcome_str)  # Convert '0' or '1' to 0 or 1
        num_samples = len(samples)
        # Assuming samples are structured as Nx2 arrays (real and imaginary parts)
        aligned_samples[qubit_idx, outcome, :num_samples, :] = samples


In [20]:
# Convert meas_outcomes to boolean for direct indexing (0 or 1)
bit_indices = meas_outcomes.astype(np.int64)  # Ensure it has the correct type for indexing


# Precompute qubit indices for each IQ index - this assumes that the mapping is static and can be precomputed
qubit_indices = np.array([simulator.qubit_mapping[IQ_idx] for IQ_idx in range(meas_outcomes.shape[1])])

# Use advanced indexing to retrieve all samples in one operation
# This conceptual step assumes alignment of your data as described; actual implementation might vary
samples = aligned_samples[qubit_indices, bit_indices, sample_indices_matrix]

# Convert samples to complex numbers
IQ_memory = samples[:, :, 0] + 1j * samples[:, :, 1]

print(IQ_memory.shape)

(100000, 63)


# Get IQ data

In [26]:
IQ_data_simul = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
print(IQ_data_simul.shape)

elapsed_time for kde_samples_needed: 0.6326889991760254
elapsed_time for kde_samples: 0.4313039779663086
elapsed_time for IQ_memory: 2.66249680519104
(100000, 63)


# Decode the data

In [27]:
import pymatching
import stim

model = simulator.stim_circ.detector_error_model(decompose_errors=False)
matching = pymatching.Matching.from_detector_error_model(model)

In [32]:
_DETAILED = False

from src import cpp_soft_info

matching = pymatching.Matching.from_detector_error_model(model)
result = cpp_soft_info.decode_IQ_fast(model, IQ_data_simul,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, nb_intervals=-1)

matching = pymatching.Matching.from_detector_error_model(model)
result_IQ_optimized = cpp_soft_info.decode_IQ_fast(model, IQ_memory,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, nb_intervals=-1)


print("num_errors IQ:", result.num_errors, "out of", len(IQ_data_simul), "shots for _RESETS =", _RESETS)
print("num_errors IQ OPTIMIZED:", result_IQ_optimized.num_errors, "out of", len(IQ_memory), "shots for _RESETS =", _RESETS)

num_errors IQ: 27667 out of 100000 shots for _RESETS = True
num_errors IQ OPTIMIZED: 0 out of 100000 shots for _RESETS = True
